In [1]:
# imports
import requests, os
import pandas as pd
from pprint import pprint
import plotly.express as px
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Foursquare

In [3]:
FOURSQUARE_KEY = os.environ['FOURSQUARE_API_KEY']

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [5]:
def fsq_place_data(latitude, longitude, radius):

    fsq_endpt_url = "https://api.foursquare.com/v3/places/search" 
    parameters = {
        'll': f"{latitude},{longitude}",
        'radius': radius,
        'categories': '13065,16000',
        'fields': 'fsq_id,name,geocodes,location,categories,distance,rating,popularity,price'
    } 
      
    headers = {
        'Accept': 'application/json',
        'Authorization': FOURSQUARE_KEY
    }
    
    fsq_res = requests.get(fsq_endpt_url, params=parameters, headers=headers)
    
    return fsq_res

In [ ]:
cityBikes_NYC = pd.read_csv('cityBikes_NYC.csv')

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [20]:
parsed_information = []
    
for index, row in cityBikes_NYC.iterrows():
    # print(row['latitude'], row['longitude'])
    # parsed_information = []

    response = fsq_place_data(latitude=row['latitude'], longitude=row['longitude'], radius=1000)
    fsq_response = response.json()

    fsq_context = fsq_response['context']

    for poi in fsq_response['results']:
        poi_info = {
            'name': poi['name'],
            'latitude': fsq_context['geo_bounds']['circle']['center']['latitude'],
            'longitude': fsq_context['geo_bounds']['circle']['center']['longitude'],
            'address': poi['location']['formatted_address'],
            'station_id': poi['categories'][0].get('id'),
            'categories': ', '.join(category['name'] for category in poi['categories']),
            'distance': poi['distance'],
        }

        try:
            poi_info['popularity'] = poi['popularity']
        except KeyError:
            poi_info['popularity'] = 0

        poi_info['distance'] = poi.get('distance') 

        parsed_information.append(poi_info)

Put your parsed results into a DataFrame

In [21]:
fsq_dataframe = pd.DataFrame(parsed_information)
fsq_dataframe.head()

,name,latitude,longitude,address,station_id,categories,distance,popularity
0,Chipotle Mexican Grill,40.795,-73.9645,"805 Columbus Ave (97th), New York, NY 10025",13145,"Fast Food Restaurant, Mexican Restaurant",108,0.990538
1,Tarr-Coyne Wild West Playground,40.795,-73.9645,"Central Park (at 93rd St), New York, NY 10025",16037,Playground,584,0.991887
2,Bánh,40.795,-73.9645,"942 Amsterdam Ave (btw 106 & 107th St), New Yo...",13032,"Cafe, Coffee, and Tea House, Sandwich Spot, Vi...",654,0.975687
3,Pio Pio 604,40.795,-73.9645,"702 Amsterdam Ave (at W 94th St.), New York, N...",13303,"Mexican Restaurant, Peruvian Restaurant, Spani...",612,0.994063
4,Mama's Too,40.795,-73.9645,"2750 Broadway (106th Street), New York, NY 10025",13064,"Pizzeria, Italian Restaurant",694,0.985078


In [22]:
fsq_dataframe.nunique()

name          2453
latitude      2009
longitude     2009
address       2571
station_id     161
categories     986
distance      1129
popularity    2613
dtype: int64

# Yelp

In [4]:
YELP_KEY = os.environ['YELP_API_KEY']

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [5]:
def get_poi_yelp(latitude_yelp, longitude_yelp, yelp_key, radius, categories_yelp):
    
    yelp_url = 'https://api.yelp.com/v3/businesses/search'

    params_yelp = {
        'latitude': latitude_yelp,
        'longitude': longitude_yelp,
        'categories': categories_yelp,
        'radius': radius
    }
   
    headers_yelp = {
        "accept": "application/json",
        "Authorization": 'Bearer '+ yelp_key
    }

    response_2 = requests.get(yelp_url, params=params_yelp, headers=headers_yelp)
    
    return response_2

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
filtered_information = []
    
for index, row in cityBikes_NYC.iterrows():
    # print(row['latitude'], row['longitude'])
    # parsed_information = []

    yelp = get_poi_yelp(latitude_yelp=row['latitude'], longitude_yelp=row['longitude'], yelp_key=YELP_KEY, radius=1000, categories_yelp='restaurants,bars,parks')
    yelp_call = yelp.json()

    yelp_coordinates = yelp_call['region']

    for business in yelp_call['businesses']:
        business_info = {
            'name': business['name'],
            'latitude': yelp_coordinates['center']['latitude'],
            'longitude': yelp_coordinates['center']['longitude'],
            'address': ', '.join(business['location']['display_address']),
            'review_count': business['review_count'],
            'rating': business['rating'],
            'categories': business['categories'][0]['title'],
            'distance': business['distance'],
            'url': business['url'],
        }

        filtered_information.append(business_info)

Put your parsed results into a DataFrame

In [8]:
yelp_dataframe = pd.DataFrame(filtered_information)
yelp_dataframe.head()

,name,latitude,longitude,address,review_count,rating,categories,distance,url
0,Jacob's Pickles,40.795,-73.9645,"509 Amsterdam Ave, New York, NY 10024",5422,4.0,Comfort Food,1313.002598,https://www.yelp.com/biz/jacobs-pickles-new-yo...
1,Arco Cafe,40.795,-73.9645,"886 Amsterdam Ave, New York, NY 10025",353,4.5,Sardinian,473.366425,https://www.yelp.com/biz/arco-cafe-new-york?ad...
2,The Calaveras,40.795,-73.9645,"949 Columbus Ave, New York, NY 10025",233,4.5,Bars,537.390215,https://www.yelp.com/biz/the-calaveras-new-yor...
3,Bareburger - Upper West Side,40.795,-73.9645,"795 Columbus Ave, Upper West Side 1, New York...",393,3.5,Burgers,131.233769,https://www.yelp.com/biz/bareburger-upper-west...
4,Saiguette,40.795,-73.9645,"935 Columbus Ave, New York, NY 10025",731,4.5,Vietnamese,472.347833,https://www.yelp.com/biz/saiguette-new-york-2?...


In [10]:
yelp_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          10000 non-null  object 
 1   latitude      10000 non-null  float64
 2   longitude     10000 non-null  float64
 3   address       10000 non-null  object 
 4   review_count  10000 non-null  int64  
 5   rating        10000 non-null  float64
 6   categories    10000 non-null  object 
 7   distance      10000 non-null  float64
 8   url           10000 non-null  object 
dtypes: float64(4), int64(1), object(4)
memory usage: 703.3+ KB


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The YELP API provided more complete data because there are two different metrics to make a decision on the how good a POI is and understand the volume of traffic that POI has received over time. The YELP API also provides more information such as phone number, website link, images for POIs.

The FOURSQUARE API only gives one metric which is really hard to understand. The floating point values are approximately the same and it's quite difficult to interpret those ratings (as least from my perspective). I would not need to google to understand number of reviews and ratings compared to popularity index. Despite, the FOURSQUARE API has the advantage that it provides a concise unique identifier for the POIs. This is useful for creating a database of locations.

Get the top 10 restaurants according to their rating

In [12]:
# yelp_dataframe[(yelp_dataframe['rating'] >= 4.5) & (yelp_dataframe['name'].str.contains('restaurants'))].head(10)
yelp_dataframe[yelp_dataframe['rating'] >= 4.5].head(10)

,name,latitude,longitude,address,review_count,rating,categories,distance,url
1,Arco Cafe,40.795000,-73.964500,"886 Amsterdam Ave, New York, NY 10025",353,4.5,Sardinian,473.366425,https://www.yelp.com/biz/arco-cafe-new-york?ad...
2,The Calaveras,40.795000,-73.964500,"949 Columbus Ave, New York, NY 10025",233,4.5,Bars,537.390215,https://www.yelp.com/biz/the-calaveras-new-yor...
4,Saiguette,40.795000,-73.964500,"935 Columbus Ave, New York, NY 10025",731,4.5,Vietnamese,472.347833,https://www.yelp.com/biz/saiguette-new-york-2?...
6,Nobody Told Me,40.795000,-73.964500,"951 Amsterdam Ave, New York, NY 10025",240,4.5,Cocktail Bars,648.275537,https://www.yelp.com/biz/nobody-told-me-new-yo...
7,Amelie,40.795000,-73.964500,"566 Amsterdam Ave, New York, NY 10024",297,4.5,French,1101.092348,https://www.yelp.com/biz/amelie-new-york-2?adj...
8,The Tang - Upper West Side,40.795000,-73.964500,"920 Amsterdam Ave, New York, NY 10025",303,4.5,Noodles,575.663991,https://www.yelp.com/biz/the-tang-upper-west-s...
9,The Grand Feast,40.795000,-73.964500,"854 Amsterdam Ave, New York, NY 10025",25,5.0,Breakfast & Brunch,428.565401,https://www.yelp.com/biz/the-grand-feast-new-y...
12,Sushi W,40.795000,-73.964500,"2673 Broadway, New York, NY 10025",177,4.5,Japanese,563.381797,https://www.yelp.com/biz/sushi-w-new-york?adju...
14,Osteria 106,40.795000,-73.964500,"53 W 106 St, New York, NY 10025",210,4.5,Italian,507.839909,https://www.yelp.com/biz/osteria-106-new-york?...
20,La Grande Boucherie,40.766741,-73.979069,"145 W 53rd St, New York, NY 10019",2480,4.5,French,481.096464,https://www.yelp.com/biz/la-grande-boucherie-n...


In [8]:
fsq_dataframe[fsq_dataframe['popularity'] >= 0.9].head(10)

,name,latitude,longitude,address,station_id,categories,distance,popularity
0,Chipotle Mexican Grill,40.795,-73.9645,"805 Columbus Ave (97th), New York, NY 10025",13145,"Fast Food Restaurant, Mexican Restaurant",108,0.990538
1,Tarr-Coyne Wild West Playground,40.795,-73.9645,"Central Park (at 93rd St), New York, NY 10025",16037,Playground,584,0.991887
2,Bánh,40.795,-73.9645,"942 Amsterdam Ave (btw 106 & 107th St), New Yo...",13032,"Cafe, Coffee, and Tea House, Sandwich Spot, Vi...",654,0.975687
3,Pio Pio 604,40.795,-73.9645,"702 Amsterdam Ave (at W 94th St.), New York, N...",13303,"Mexican Restaurant, Peruvian Restaurant, Spani...",612,0.994063
4,Mama's Too,40.795,-73.9645,"2750 Broadway (106th Street), New York, NY 10025",13064,"Pizzeria, Italian Restaurant",694,0.985078
5,Osteria 106,40.795,-73.9645,"53 W 106th St (Btwn Manhattan and Columbus), N...",13236,"Italian Restaurant, Mediterranean Restaurant",482,0.965759
6,Silver Moon Bakery,40.795,-73.9645,"2740 Broadway (at W 105th St), New York, NY 10025",13002,"Bakery, Restaurant",676,0.971433
7,The Ravine,40.795,-73.9645,"105th St, New York, NY 10028",16046,Scenic Lookout,715,0.978152
8,Makana Hawaiian BBQ & Poke,40.795,-73.9645,"161 W 106th St (at Amsterdam Ave), New York, N...",13026,"BBQ Joint, Hawaiian Restaurant",591,0.979252
9,North Woods,40.795,-73.9645,"New York, NY 10028",16032,Park,836,0.964528
